<a href="https://colab.research.google.com/github/aidanjwest/Digital-Humanities-Project.-/blob/main/Pdf_creation_and_text_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### Part I. Creates artificial data using a markov word generator, and turns it into a pdf vaguely resembling the FCO documents that the final model will be used on. This pdf is then used in the colab doc.
#This code was initially compiled on a local jupyter notebook, but transferred to colab for sake of consistency. 

import string
!pip install fpdf
from fpdf import FPDF 

import random
from itertools import repeat, chain

!pip install essential_generators

from essential_generators import DocumentGenerator
import re

from itertools import repeat, chain

The code below creates the training data. A Markov chain generator is used, along with some built-in python formatting to underline, and capitalize random parts of the text. This is then written into a text file. 

In [ ]:

def underline(s):
    return ''.join(chain.from_iterable(zip(s, repeat("\u0332"))))

gen = DocumentGenerator()

model_text = ""
sentence_count = 0

while sentence_count < 2500:
    sentence = str(gen.sentence())
    
    if sentence_count % 2 == 0:
        model_text += sentence + " "
        sentence_count += 1
    else:
        model_text += sentence.upper() + " "
        sentence_count += 1

model_text = ascii(model_text)

re.sub(' +', ' ', model_text)

with open("model_text.txt", 'w', encoding="utf-8") as f:
    f.write(model_text)

with open("model_text.txt", 'r', encoding="utf-8") as f:
    content = f.read()

The code below uses that text file to create the pdf. The font is Courier, size 10 so as to resemble the FCO typewriter style. The colour (223, 229, 227) is vaguely grey-ish, for the same purpose. The style is also formatted to look the same, with the margins and titles set in the same style. 


In [ ]:

title = 'Confidential'

class PDF(FPDF):
    def header(self):
        self.set_font('Courier', 'B', 15)
        w = self.get_string_width(title) + 6
        self.set_x((210 - w) / 2)
        self.set_draw_color(223, 229, 227)
        self.set_fill_color(223, 229, 227)
        #self.set_text_color(220, 50, 50)
        self.set_line_width(1)
        self.cell(w, 9, title, 1, 1, 'C', True)
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Courier', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def chapter_title(self, num, label):
        #self.set_font('Courier', 12)
        self.set_fill_color(223, 229, 227)
        self.cell(0, 6, 'Chapter %d: %s' % (num, label), 0, 1, 'L', 1)
        self.ln(4)

    def chapter_body(self, name):
        with open(name, 'rb') as fh:
            txt = fh.read().decode("latin-1")
        # Times 12
        self.set_font('Courier', '', 11)
        self.set_fill_color(223, 229, 227)           
        self.multi_cell(170, 5, txt, align = 'l', fill = True)
        self.ln()
        # Mention in italics
        self.set_font('', 'I')
        self.cell(0, 5, '(end of excerpt)')

    def print_chapter(self, num, title, name):
        self.add_page()
        self.chapter_title(num, title)
        self.chapter_body(name)

pdf = PDF()
pdf.set_title(title)
pdf.set_author('The FCO')
pdf.print_chapter(1, "Looking Good", 'model_text.txt')
pdf.output('Model_text.pdf', 'F')



This code then automates the process of filling out the editable html that kraken produced. 

In [ ]:

print(content[6610:6620])

import requests 
from bs4 import BeautifulSoup

from urllib.request import urlopen as uReq
import os
import re

with open('more_lines.txt', 'r') as file:
    content = file.readlines()
    print(len(content))

soup = BeautifulSoup(open(r'C:\Users\Barry Tuba\Documents\Year 4 loool\Digital Humanities\Finding_Sarah.html'), 'lxml')
test_containers = soup.find_all('li') 

print(type(test_containers), "length:", len(test_containers))
print(len(content))

silver_surfer = [tag for tag in test_containers if tag.find("a", class_ = 'page_anchor') == None]

for tag, sentence in zip(silver_surfer, content):
    tag.append(sentence)


with open("example_modified.html", "wb") as f_output:
    f_output.write(soup.prettify("utf-8"))  
